In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.svm import SVC

In [2]:
samples = pd.read_table("linear_train.txt", sep=',', header=None)
test = pd.read_table("linear_test.txt", header=None)
print(test.shape[0])

188920


In [3]:
samples = samples.sample(frac=1)
N = samples.shape[0]
X_train = samples[0]
y_train = samples[1]
# X = [x + ' ' for x in X]
X_train = np.array(X_train)
X_train = np.array(X_train)

In [4]:
X_test = test[0]

In [5]:
X = list(X_train) + list(X_test)

In [7]:
cnt_vec = CountVectorizer(analyzer='char', ngram_range=(2, 9), binary=True)

In [8]:
X_vec = cnt_vec.fit_transform(X)

In [9]:
X_vec_train = X_vec[:N]

In [10]:
y_ones = np.array([yy == 1 for yy in y_train]).sum() / len(y_train)
y_zeros = np.array([yy == 0 for yy in y_train]).sum() / len(y_train)

In [12]:
def binary_auc(y, y_pred):
    for i in range(len(y)):
        if y_pred[i] > y_zeros: 
            y_pred[i] = 1
        else:
            y_pred[i] = 0
    return roc_auc_score(y, y_pred)

In [11]:
scorer = make_scorer(score_func=roc_auc_score)

In [176]:
model = LogisticRegression(random_state=95, max_iter=200, penalty='l2', C=1.19)

In [177]:
for C in [10000]:
    model = LogisticRegression(random_state=95, max_iter=200, penalty='l2',
                               C=C, class_weight={0: y_zeros, 1: y_ones})
    accuracy = cross_val_score(model, X_vec_train, y_train, scoring=scorer)
    print(accuracy)

[ 0.72982779  0.73497776  0.72775974]


In [13]:
model = LinearRegression()

In [14]:
accuracy = cross_val_score(model, X_vec_train, y_train, scoring=scorer)
print(accuracy)

[ 0.82626093  0.83089089  0.82366856]


In [150]:
model = SVC()

In [151]:
accuracy = cross_val_score(model, X_vec_train, y_train, scoring=scorer)
print(accuracy)

[ 0.5  0.5  0.5]


In [15]:
model.fit(X_vec_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [50]:
# for C in [1.18, 1.19, 1.2, 1.21, 1.22, 1.23]:
#     model = LogisticRegression(random_state=95, max_iter=200, penalty='l2', C=C)   
#     accuracy = cross_val_score(model, X_vec, y)
#     print(accuracy.mean())

In [20]:
X_test_vec = X_vec[N:]
preds = model.predict(X_test_vec)

In [21]:
submission = pd.read_csv("linear_ans_example.txt")
print(len(preds), submission.shape[0])
submission['Answer'] = preds
submission['Answer'] = submission['Answer'].map(lambda x: x if x > 0 else 0.0)
submission.head(5)
submission.to_csv("baseline_submission.tsv", sep=',', index=False)

188920 188920
